# 1 面向对象

## 1.1 基本用法

面向对象编程是构建复杂软件系统必不可少的编程范式，他有四大特性：**抽象、封装、继承和多态**。在Python中，面向对象同样是基设施。

```python
class Document():
    def __init__(self, title, author, context):
        print('init function called')
        self.title = title
        self.author = author
        self.__context = context # __开头的属性是私有属性

    def get_context_length(self):
        return len(self.__context)

    def intercept_context(self, length):
        self.__context = self.__context[:length]
```

这就是面向对象基本的语法，`__init__`是类的初始化方法，它的第一个参数永远是`self`，后面接其他属性。

In [1]:
class Document():
    def __init__(self, title, author, context):
        print('init function called')
        self.title = title
        self.author = author
        self.__context = context # __开头的属性是私有属性

    def get_context_length(self):
        return len(self.__context)

    def intercept_context(self, length):
        self.__context = self.__context[:length]
        
harry_potter_book = Document('Harry Potter', 
                             'J. K. Rowling', 
                             '... Forever Do not believe any thing is capable of thinking independently ...')

print(harry_potter_book.title)
print(harry_potter_book.author)
print(harry_potter_book.get_context_length())

harry_potter_book.intercept_context(10)

print(harry_potter_book.get_context_length())

print(harry_potter_book.__context) # 将会报错AttributeError，因为__context是私有属性

init function called
Harry Potter
J. K. Rowling
77
10


AttributeError: 'Document' object has no attribute '__context'

## 1.2 @classmethod @staticmethod

类函方法`@classmethod`和成员方法类似，都和类自身有一定的关系，前者第一个参数是`cls`，后者第一个参数是`self`，类函数最大的使用场景是提供不同的初始化方法。

静态方法`@staticmethod`和普通的函数没有区别，不需要传入实例和类，主要为了组织代码，比如配置一些环境变量，修改其他类的属性等。

```python
class A(object):
    def foo(self, x):
        print "executing foo(%s, %s)" % (self, x)

    @classmethod
    def class_foo(cls, x):
        print "executing class_foo(%s, %s)" % (cls, x)

    @staticmethod
    def static_foo(x):
        print "executing static_foo(%s)" % x    

a = A()
```

在Python中，一般约定用大写字母表示常量，在类中也是这样，我们可以在类中使用 `self.WELCOME_STR` ，或者在类外使用 `Document.WELCOME_STR` ，来表达这个字符串。



In [2]:
class A(object):
    def foo(self, x):
        print("executing foo {}, {})".format(self, x))

    @classmethod
    def class_foo(cls, x):
        print("executing class_foo {}, {}".format(cls, x))

    @staticmethod
    def static_foo(x):
        print("executing static_foo {}".format(x))

a = A()

a.foo(10)

A.class_foo(11)
a.class_foo(12) # 这种方式将a的类对象隐式传入

A.static_foo(21)
a.static_foo(22)

executing foo <__main__.A object at 0x1032c8a60>, 10)
executing class_foo <class '__main__.A'>, 11
executing class_foo <class '__main__.A'>, 12
executing static_foo 21
executing static_foo 22


## 1.3 元类metaclass

meta-class的meta这个词根，起源于希腊语词汇meta，包含下面两种意思：

1. “Beyond”，例如技术词汇metadata，意思是描述数据的超越数据；
2. “Change”，例如技术词汇metamorphosis，意思是改变的形态。

In [24]:
# Python 2/3 相同部分

class Loader(object):
    pass

class YAMLObjectMetaclass(type):
  def __init__(cls, name, bases, kwds):
    super(YAMLObjectMetaclass, cls).__init__(name, bases, kwds)
    if 'yaml_tag' in kwds and kwds['yaml_tag'] is not None:
      cls.yaml_loader.add_constructor(cls.yaml_tag, cls.from_yaml)
  # 省略其余定义

# Python 3
class YAMLObject(metaclass=YAMLObjectMetaclass):
  yaml_loader = Loader
  # 省略其余定义

# Python 2
class YAMLObject(object):
  __metaclass__ = YAMLObjectMetaclass
  yaml_loader = Loader
  # 省略其余定义

In [27]:
import yaml

document = """
    a: 1
    b:
        c: 2
        d: 3
"""
print(yaml.safe_load(document))

yaml.safe_dump(yaml.safe_load(document))

{'a': 1, 'b': {'c': 2, 'd': 3}}


'a: 1\nb:\n  c: 2\n  d: 3\n'

In [28]:
class Monster(yaml.YAMLObject):
  yaml_tag = u'!Monster'
  def __init__(self, name, hp, ac, attacks):
    self.name = name
    self.hp = hp
    self.ac = ac
    self.attacks = attacks
  def __repr__(self):
    return "%s(name=%r, hp=%r, ac=%r, attacks=%r)" % (
       self.__class__.__name__, self.name, self.hp, self.ac,      
       self.attacks)

yaml.full_load("""
!Monster
name: Walker
hp: [2,3]
ac: 32.23
attacks: [Ac, Df]
""")

Monster(name='Walker', hp=[2, 3], ac=32.23, attacks=['Ac', 'Df'])

# 2 对象比较
## 2.1 等值测试

在Python中，对象的比较有两种方式，即`==` 和 `is`，它们之间的区别是：
- ==，对象的值之间的比较，不同类型之间比较设计重载，效率会降低；
- is，对象标识之间的比较，也就是内存地址，通过`id(obj)`获取标识，效率高。

在实际工作中，前者的使用频率高于后者，因为我们更加关心变量的值是否相同。但是，在某些情况下，后者也常常会被使用，就是对None的判断。

```python
if a is None:
      ...

if a is not None:
      ...
```

In [8]:
int_10 = 10
another_int_10 = 10
print(int_10 == another_int_10)
print(int_10 is another_int_10)

int_1000 = 1000
another_int_1000 = 1000
print(int_1000 == another_int_1000)
print(int_1000 is another_int_1000)

True
True
True
False


上面这个奇怪的现象，是因为：

> 出于对性能优化的考虑，Python内部会对-5到256的整型维持一个数组，起到一个缓存的作用。这样，每次你试图创建一个-5到256范围内的整型数字时，Python都会从这个数组中返回相对应的引用，而不是重新开辟一块新的内存空间。

所以，是因为1000超出了这个范围，Python为int_1000和another_int_1000开辟了两块内存空间，这才导致它们的标识不同。

## 2.2 拷贝

拷贝对象在每个PL中都存在，分为深拷贝和钱拷贝，在不同PL中它们的意义也不尽相同。

### 2.2.1 浅拷贝
在Python中，对象的浅拷贝是指：**重新分配一块内存空间给新对象，并且里面的子对象（属性、方法等）全部引用原对象的**。所以，如果原对象是不可变的还好，但如果是可变的，则会带来一些副作用。

下面都是浅拷贝的例子。

In [13]:
l1 = [1, 2, 3]
l2 = list(l1)
print(f"(l1 == l2): {l1 == l2}, l1 is l2: {l1 is l2}")

s1 = set([1, 2, 3])
s2 = set(s1)
print(f"(s1 == s2): {s1 == s2}, s1 is s2: {s1 is s2}")


# 可变对象还可以通过切片完成浅拷贝
l1 = [1, 2, 3]
l2 = l1[:]
print(f"(l1 == l2): {l1 == l2}, l1 is l2: {l1 is l2}")

(l1 == l2): True, l1 is l2: False
(s1 == s2): True, s1 is s2: False
(l1 == l2): True, l1 is l2: False


另外，也可以copy模块中的copy方法实现浅拷贝。

In [15]:
import copy

l1 = [1, 2, 3]
l2 = copy.copy(l1)
print(f"(l1 == l2): {l1 == l2}, l1 is l2: {l1 is l2}")

(l1 == l2): True, l1 is l2: False


不过，需要注意的是，对于元组，使用tuple()或者切片操作符':'不会创建一份浅拷贝，相反，它会返回一个指向相同元组的引用。

In [16]:
t1 = (1, 2, 3)
t2 = tuple(t1)
print(f"(t1 == t2): {t1 == t2}, t1 is t2: {t1 is t2}")

(t1 == t2): True, t1 is t2: True


当然，浅拷贝存在的副作用也不可忽视。比如对于一个嵌套列表对象，原对象和新对象因为共享一个内层列表，当其中一个修改时，另一个也会受到影响。

In [19]:
l1 = [[1, 2], (30, 40)]
l2 = list(l1)
l1.append(100)
l1[0].append(3)
print(f"l1 is {l1}\nl2 is {l2}")

l1[1] += (50, 60)
print(f"l1 is {l1}\nl2 is {l2}")

l1 is [[1, 2, 3], (30, 40), 100]
l2 is [[1, 2, 3], (30, 40)]
l1 is [[1, 2, 3], (30, 40, 50, 60), 100]
l2 is [[1, 2, 3], (30, 40)]


### 2.2.2 深拷贝

所谓深度拷贝，是指重新分配一块内存，创建一个新的对象，并且将原对象中的元素，以递归的方式，通过创建新的子对象拷贝到新对象中。因此，新对象和原对象没有任何关联。

Python中以copy.deepcopy()来实现对象的深度拷贝。

In [20]:
import copy
l1 = [[1, 2], (30, 40)]
l2 = copy.deepcopy(l1)
l1.append(100)
l1[0].append(3)
print(f"l1 is {l1}\nl2 is {l2}")

l1 is [[1, 2, 3], (30, 40), 100]
l2 is [[1, 2], (30, 40)]


不过，深度拷贝也不是完美的，往往也会带来一系列问题。如果被拷贝对象中存在指向自身的引用，那么程序很容易陷入无限循环。

In [21]:
x = [1]
x.append(x)

y = copy.deepcopy(x)

print(f"x is {x}, y is {y}")

x is [1, [...]], y is [1, [...]]


上面这个例子，列表x中有指向自身的引用，因此x是一个无限嵌套的列表。但是我们发现深度拷贝x到y后，程序并没有出现stack overflow的现象。这是为什么呢？

其实，这是因为深度拷贝函数deepcopy中会维护一个字典，记录已经拷贝的对象与其ID。拷贝过程中，如果字典里已经存储了将要拷贝的对象，则会从字典直接返回，看相对应的源码就能明白：

```python
def deepcopy(x, memo=None, _nil=[]):
    """Deep copy operation on arbitrary Python objects.
    	
	See the module's __doc__ string for more info.
	"""
	
    if memo is None:
        memo = {}
    d = id(x) # 查询被拷贝对象x的id
	y = memo.get(d, _nil) # 查询字典里是否已经存储了该对象
	if y is not _nil:
	    return y # 如果字典里已经存储了将要拷贝的对象，则直接返回
        ...    
```

# 3 赋值与传递

在Python中：

- 变量的赋值，只是表示让变量指向了某个对象，并不表示拷贝对象给变量；而一个对象，可以被多个变量所指向。
- 可变对象（列表，字典，集合等等）的改变，会影响所有指向该对象的变量。
- 对于不可变对象（字符串、整型、元组等），所有指向该对象的变量的值总是一样的，也不会改变。但是通过某些操作（+=等等）更新不可变对象的值时，会返回一个新的对象。
- 变量可以被删除，但是对象无法被删除。

至于函数参数的传递，这里首先引用Python官方文档中的一段说明：

> Remember that arguments are passed by assignment in Python. Since assignment just creates references to objects, there’s no alias between an argument name in the caller and callee, and so no call-by-reference

准确地说，Python的参数传递是赋值传递 （pass by assignment），或者叫作对象的引用传递（pass by object reference）。

具体来说，对于不可变对象，简单的执行函数并不会对调用前的变量产生影响。但如果参数为可变类型，比如list，并且函数内有修改参数的逻辑，那么参数本身也会受到影响。

In [23]:
def my_func1(b):
    b = 2

a = 1
my_func1(a)
print(a)

def my_func1(b):
    b = 2
    return b

a = 1
a = my_func1(a)
print(a)

def my_func3(l2):
    l2.append(4)

l1 = [1, 2, 3]
my_func3(l1)
print(l1)

1
2
[1, 2, 3, 4]
